In [1]:
# https://medium.com/@a173823/web-scraping-on-shopee-com-6180979eff78
# Regulatron01!
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep
from collections import Counter
from tqdm import tqdm
import time
import re

import pandas as pd

# import threading

# import numpy as np
# from numpy import nan
# import re
# import concurrent.futures
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math

In [2]:
def converter_dict(dict_original):
    dict_convertido = {}
    for chave, valor in dict_original.items():
        if '|' in valor:
            lista_valor = valor.split('|')
            dict_convertido[chave] = lista_valor
        else:
            dict_convertido[chave] = valor
    return dict_convertido

def testar_palavras(texto_pesquisado, yes_words, no_words)-> bool:
    
    """
    Testa se as palavras de yes-words e no-words estão dentro de texto_pesquisado.

    Args:
        texto_pesquisado (str): Texto que deve vir da página web. Pode mesclar o título com a descrição do produto.
        yes_words (list): lista de palavras que fazem a função retornar True.
        no_words (list): lista de palavras que fazem a função retornar False.

    Returns:
        Bolean:
    """   
    contagem_no_words = 0
    texto_pesquisado = texto_pesquisado.lower()
    
    for palavra in yes_words:
        print('========================== yes_words ')
        if palavra in texto_pesquisado:
            return True
    
    for palavra in no_words:
        print('========================== no words ')
        if palavra in texto_pesquisado:
            contagem_no_words+= 1
    
    print ('total:', contagem_no_words)
    if contagem_no_words > 0:
        return False
    else:
        return True    

In [ ]:
# create object for chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                              options = chrome_options)  
driver.implicitly_wait(5)
actions = ActionChains(driver)

# Maximiza a janela
driver.maximize_window()
# navegar até a shopee
driver.get('https://shopee.com.br/')
time.sleep(3)
elements = driver.find_elements(By.TAG_NAME, 'button')

# clica em aceitar todos os cookies
for e in elements:
    if e.text == 'Aceitar todos os cookies':
        actions.move_to_element(e)
        actions.perform()
        e.click()
        break
        

# clica em algum lugar da página
element = driver.find_element(By.ID, 'main')
actions.move_to_element(element).click().perform()
time.sleep(3)

# clicar para entrar
e = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Entre')[0]
actions.move_to_element(e).click().perform()

# fazer login
telefone = '71999243010'
senha    = 'Regulatron01!'

campo_telefone = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[0]
campo_senha = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[1]

actions.move_to_element(campo_telefone).click().perform()
for i in telefone:
    campo_telefone.send_keys(i)
    time.sleep(0.5)

actions.move_to_element(campo_senha).click().perform()
for i in senha:
    campo_senha.send_keys(i)
    time.sleep(0.5)

# clicar para entrar
elements = driver.find_elements(By.TAG_NAME, 'button')
for e in elements:
    if e.text == 'ENTRE':
        actions.move_to_element(e).click().perform()
        break
        
time.sleep(3)

# clica em algum lugar da página
e = driver.find_element(By.CSS_SELECTOR, 'body')
actions.move_to_element(e).click().perform()
time.sleep(3)
search_box = driver.find_element(By.CLASS_NAME, 'shopee-searchbar-input__input')
actions.move_to_element(search_box).click().perform()
time.sleep(0.5)

# pesquisar
limite_de_produtos = 0
set_produtos = set() 
query = 'tvbox'
time.sleep(0.5)

for i in query:
    search_box.send_keys(i)
    time.sleep(0.5)

header = driver.find_element(By.TAG_NAME,'header')
button = header.find_elements(By.TAG_NAME,'button')[0]
actions.move_to_element(button).click().perform()
time.sleep(3)



## percorrer páginas
try:
    ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
except:
    ultima_pagina = 1

for i in range(ultima_pagina):
    if ( len(set_produtos) == limite_de_produtos ) and ( limite_de_produtos > 0 ) :
        break
    else:
        print('pesquisando pág', i+1, 'de', ultima_pagina)

        # rolagem para esperar carregar os produtos
        for i in range(50):
            try:
                driver.execute_script("window.scrollBy(0, 100)")
                time.sleep(0.5)
            except:
                break


        time.sleep(3)
        div_links = driver.find_element(By.CSS_SELECTOR, '#main > div > div.dYFPlI > div > div > div.sdzgsX > div.shopee-search-item-result > div.row.shopee-search-item-result__items')
        a_elements = div_links.find_elements(By.TAG_NAME, 'a')

        for a in a_elements:
            set_produtos.add(a.get_attribute('href'))

        button = driver.find_elements(By.CLASS_NAME, 'shopee-icon-button--right')[0]
        actions.move_to_element(button).click().perform()
        time.sleep(3)

        
dict_produtos = { # vai guardar todas as informações dos produtos
    'produto_pesquisado' : [],
    'titulo_produto' : [],
    'id_vendedor': [],
    'quantidade' : [],
    'localizacao' : [],
    'marca' : [],
    'preco_minimo': [],
    'preco_maximo': [],
    'preco_promo_minimo': [],
    'preco_promo_maximo': [],
    'detalhes_produto' : [],
    'quantidade': [],
    'descricao': [] ,
    'homologado': [],
    'plataforma' : [],
    'url' : [],
    'modelo' : []
}

faltam = len(set_produtos)

print('pesquisando produtos')

for link in set_produtos:
    print('faltam', faltam, 'produtos')
    faltam = faltam -1
    driver.get(link)
    
    time.sleep(3)

    for i in range(10):
        try:
            driver.execute_script("window.scrollBy(0, 100)")
            time.sleep(0.5)
        except:
            break

    try:
        titulo_produto = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                            ).text
    except:
        titulo_produto = '-'

    try:
        id_vendedor    = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                            ).text
    except:
        id_vendedor    = '-'

    try:      
        qtde_vendida = driver.find_element(By.XPATH,
                                           '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                          ).text
    except:
        qtde_vendida = '0'

    try:
        localizacao = driver.find_element(By.XPATH,
                                            '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                         ).text
    except:
        localizacao = ''

    try:
        preco = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                                   ).text

        if preco.find('-'):
            preco = preco.split('-')
            preco_minimo = preco[0][2:]
            preco_maximo = preco[-1][3:]
        else:
            preco_minimo = preco[2:]
            preco_maximo = '-'
    except:
        preco_minimo = '-'
        preco_maximo = '-'

    try:
        preco_promo = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                                   ).text

        if preco_promo.find('-'):
            preco_promo = preco_promo.split('-')
            preco_promo_minimo = preco_promo[0][2:]
            preco_promo_maximo = preco_promo[-1][3:]
        else:
            preco_promo_minimo = preco_promo[2:]
            preco_promo_maximo = '-'

    except:
            preco_promo_minimo = '-'
            preco_promo_maximo = '-'


    try:
        detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                              ).get_attribute('innerHTML')
        detalhes_produto = detalhes_produto.replace('</a>','|</a>')

        lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

        # criar um objeto Beautiful Soup a partir do HTML do div
        soup = BeautifulSoup(detalhes_produto, 'html.parser')

        # remover todas as tags <label> e svg
        for label in soup.find_all(['label','svg']):
            label.extract()

        for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
            lista_valores.unwrap()

        # seleciona todas as tags <div> e extrai o texto
        lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
        detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
        detalhes_produto = converter_dict(detalhes_produto)

    except:
        detalhes_produto = '-'

    try:
        descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
    except:
        descricao = '-'

    try:
        quantidade = detalhes_produto['Estoque']

    except:
        quantidade = '0'

    try:
        modelo = detalhes_produto['Modelo']

    except:
        modelo = '-'
        
    try:
        marca = detalhes_produto['Marca']

    except:
        marca = '-'

    # if testar_palavras(f'{titulo_produto} {descricao} {detalhes_produto.values()}', yes_words, no_words):
    #     homologado = 'CANDIDATO'
    # else:
    #     homologado = ''
    homologado = ''

    dict_produtos['produto_pesquisado'].append(query)
    dict_produtos['titulo_produto'].append(titulo_produto)
    dict_produtos['id_vendedor'].append(id_vendedor)
    dict_produtos['quantidade'].append(quantidade)
    dict_produtos['localizacao'].append(localizacao)
    dict_produtos['marca'].append(marca)
    dict_produtos['preco_minimo'].append(preco_minimo)
    dict_produtos['preco_maximo'].append(preco_maximo)
    dict_produtos['preco_promo_minimo'].append(preco_promo_minimo)
    dict_produtos['preco_promo_maximo'].append(preco_promo_maximo)
    dict_produtos['detalhes_produto'].append(detalhes_produto)
    dict_produtos['descricao'].append(descricao)
    dict_produtos['homologado'].append(homologado)
    dict_produtos['plataforma'].append('Shopee')
    dict_produtos['url'].append(link)
    dict_produtos['modelo'].append(modelo)



df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}.csv', encoding='utf-8', index = False, sep =';')

pesquisando pág 0 de 17
pesquisando pág 1 de 17
pesquisando pág 2 de 17
pesquisando pág 3 de 17
pesquisando pág 4 de 17
pesquisando pág 5 de 17
pesquisando pág 6 de 17
pesquisando pág 7 de 17
pesquisando pág 8 de 17
pesquisando pág 9 de 17
pesquisando pág 10 de 17
pesquisando pág 11 de 17
pesquisando pág 12 de 17
pesquisando pág 13 de 17
pesquisando pág 14 de 17
pesquisando pág 15 de 17
pesquisando pág 16 de 17
pesquisando produtos
faltam 1020 produtos
faltam 1019 produtos
faltam 1018 produtos
faltam 1017 produtos
faltam 1016 produtos
faltam 1015 produtos
faltam 1014 produtos
faltam 1013 produtos
faltam 1012 produtos
faltam 1011 produtos
faltam 1010 produtos
faltam 1009 produtos
faltam 1008 produtos
faltam 1007 produtos
faltam 1006 produtos
faltam 1005 produtos
faltam 1004 produtos
faltam 1003 produtos
faltam 1002 produtos
faltam 1001 produtos
faltam 1000 produtos
faltam 999 produtos
faltam 998 produtos
faltam 997 produtos
faltam 996 produtos
faltam 995 produtos
faltam 994 produtos
fal

In [55]:
df.head()

,produto_pesquisado,titulo_produto,id_vendedor,quantidade,localizacao,marca,preco_minimo,preco_maximo,preco_promo_minimo,preco_promo_maximo,detalhes_produto,descricao,homologado,plataforma,url,modelo
0,tvbox,Indicado\nG2 4K Hdmi Sem Fio DONGLE Google Chr...,aloha.br,789,,-,"97,98","7,98","45,20","5,20","{'Categoria': ['Shopee', 'Celulares e Disposit...",Descrição do produto em inglês após português\...,,Shopee,9,-
1,tvbox,Estoque Pronto ! D905 Set-Top Box Android 7.1 ...,lixia202210.br,3828,China Continental,-,"108,00","08,00",-,-,"{'Categoria': ['Shopee', 'Computadores e Acess...",: Lembrete gentil: plugues do Brasil escolhem ...,,Shopee,9,-
2,tvbox,UEB H96 Max V12 Android TV Set Top Box Media P...,-,0,,-,"305,99","398,99","152,09","195,99",-,-,,Shopee,9,-
3,tvbox,Indicado\nAdaptador Dongle Chromecast G2 TV St...,aloha.br,269,,-,"120,00","20,00","44,99","4,99","{'Categoria': ['Shopee', 'Celulares e Disposit...",Descrição do Produto:\n\nTrês Protocolo de Tra...,,Shopee,9,-
4,tvbox,TV Inteligente 2023 Android 10.0 4K 5G 16GB 25...,topelectronicmall.br,1998,,-,"229,00","279,00","149,00","189,00","{'Categoria': ['Shopee', 'Eletroportáteis', 'O...",,,Shopee,9,-
